# Describe

The Arc in ArcGIS stands for "always right-click". That's how you get information about the stuff you're working with. But Python scripts can't right-click. They don't have a mouse, or even fingers. You still need the same information, but you're going to need to get it a different way. 

The goal of this notebook is to help you use the Describe function to get information you can use in your script.